# Firts Analysis

### quickly functions

In [1]:
def magnitude(vector): 
    return np.sqrt(sum(pow(element, 2) for element in vector))

In [2]:
def hist(x, bins, xtitle, ytitle = None, title = None):
    
    plt.figure(figsize=(8,6))
    sn.histplot(x, bins = bins)
    plt.xlabel(xtitle)
    
    if ytitle == None:
        plt.ylabel('events')
    else:
        plt.ylabel(ytitle)
    if title == None:
        plt.title(xtitle + ' vs ' + ytitle)
    else:
        plt.title(title)
        
    plt.yscale('log')

In [3]:
def hist2D(x_var, y_var, bins, xtitle, ytitle):
    plt.figure(figsize=(8,6))
    sn.histplot(x = x_var, y = y_var, bins = bins)
    plt.xlabel(xtitle)
    plt.ylabel(ytitle)
    #plt.xlim(-800,500)
    plt.show()

------------------------------------

In [4]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from tqdm import tqdm
from mpl_toolkits import mplot3d
from numpy import array, where, shape, reshape
np.set_printoptions(threshold=1000)

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


### Importar archivos

In [3]:
file = uproot.open("/sno/py_out1.root")

### Explorar TTrees

In [4]:
branches = file.keys()
branches

['T;7', 'T;6', 'pmt;1']

In [5]:
data1 = file['T;7']
data1.keys()

['evtid',
 'mc_position',
 'mc_momentum',
 'position',
 'momentum',
 'hit_pmtid',
 'hit_pmttime',
 'hit_residual',
 'hit_type']

In [6]:
evtid = np.array(data1['evtid'])
mc_position = np.array(data1['mc_position'])
position = np.array(data1['position'])
hit_pmtid = np.array(data1['hit_pmtid'])
hit_pmttime = np.array(data1['hit_pmttime'])
hit_residual = np.array(data1['hit_residual'])
hit_type = np.array(data1['hit_type'])

#### Nº of events

In [7]:
len(evtid)

14618962

### Verify if plots make sence

In [320]:
var = hit_residual

hist(var, 100, 'hit_residual')

In [9]:
where(hit_residual<-8000)  

(array([    8496,    17942,    24294,    54177,    62623,    71097,
           96783,   109677,   133324,   177416,   222694,   241540,
          242989,   267741,   286499,   297883,   297884,   322279,
          323721,   336923,   358067,   366261,   375682,   376907,
          383911,   406250,   422409,   451001,   464363,   465126,
          465946,   468328,   481663,   505454,   568770,   578153,
          582870,   603927,   607919,   626504,   656223,   676579,
          718026,   734900,   743677,   783994,   783995,   837603,
          841501,   850630,   860903,   867864,   873123,   892160,
          892161,   895359,   903470,   922094,   932258,   951231,
          954963,   984849,   988796,  1025885,  1033181,  1072282,
         1082607,  1099407,  1110264,  1131202,  1157636,  1210117,
         1242447,  1250070,  1267689,  1268553,  1293839,  1305322,
         1313635,  1320564,  1335565,  1355148,  1385422,  1390936,
         1404421,  1407461,  1465975,  1468118, 

In [350]:
#take one of the eventsID with high negative time residual value
evtid[17942]

23

#### pmtID giving hits with time residual too negative! (t_res about -10.000 and -600)

In [312]:
hist2D(hit_residual, hit_pmtid, 100, 'hit_residual','hit_pmtid')
plt.xlim(-800,500)

(-800.0, 500.0)

### Extract PMT info.
PMT info. is relative to its standar positions, identificators, etc. They dont correspond to hits of events!

In [276]:
pmt_info = file['pmt;1']
pmt_info.keys()

['pmt_id', 'pmt_pos_xyz', 'pmt_pos_sph', 'pmt_type']

In [277]:
pmt_id = array(pmt_info['pmt_id'])
pmt_pos_xyz = array(pmt_info['pmt_pos_xyz'])
pmt_pos_sph = array(pmt_info['pmt_pos_sph'])
pmt_type = array(pmt_info['pmt_type'])

#Now, pass to array--------------

#pmt_id = np.array(pmt_id)
#pmt_pos_xyz = np.array(pmt_pos_xyz)
#pmt_pos_sph = np.array(pmt_pos_sph)
#pmt_type = np.array(pmt_type)

In [278]:
np.shape(pmt_pos_xyz)

(9728, 3)

In [279]:
np.shape(pmt_pos_sph)

(9728, 3)

In [280]:
np.shape(pmt_type)

(9728,)

### Clean PMT type (just pmt_type = 1 || pmt_type = 7)
Get the list of pmtID which are valid in the PMT info

In [281]:
type_ = array([1,7])
valid_index_pmt_type = np.array([],np.int32)
clean_pmt_id = np.array([],np.int32) #pmtID which are valid

#valid_index_pmt_type_list = [] # List of Indices of valids pmtID (util solo en pmt info, no en registro de eventos -> guardo el array, no la lista!!)

for i_type in tqdm(type_):
    p = where(pmt_type == i_type)[0] #look for indices ef valid PMT
    #if i_type == 7:
    #    print(p)
    valid_index_pmt_type = np.concatenate((valid_index_pmt_type,p))
    
    #valid_index_list.append(p)
    #print(valid_index_list)
    #valid_index_pmt_type = valid_index_list[0][0] #extact the np.array
#print(valid_index_pmt_type_list)

for valid_i in tqdm(valid_index_pmt_type):
    clean_pmt_id = np.append(clean_pmt_id,pmt_id[valid_i])
    
print(clean_pmt_id)
len(clean_pmt_id)

100%|███████████████████████████████████████████████████████████████████████████| 9394/9394 [00:00<00:00, 67143.17it/s]

[   1    2    3 ... 3536 3856 8163]


9394

# Event Analysis

Lest choose one event and see what we can see

### Useful variables

In [369]:
event = 11 #EventID = 14 #EventID = 11,23,... is problematic
condition = (evtid == event) #which event ID is true 

pmtid_ev = np.extract(condition, hit_pmtid) #Indices for which elements are True in eventID then extract the hit_pmtID
time_res_ev = np.extract(condition, hit_residual)
pmttime_ev = np.extract(condition, hit_pmttime)
N_events = len(pmtid_ev)

#special trait to extract positions...
pre_mc_position_ev = np.array([]) #Monte Carlo position (pre bc need to be reshaped - coordinates are all aligned)
pre_position_ev = np.array([]) 

mc_radius_ev = np.array([])  #Monte Carlo radius
radius_ev = np.array([]) #reconstructed radius

for i in where(condition)[0]:
    pre_mc_position_ev = np.append(pre_mc_position_ev, mc_position[i])
    pre_position_ev = np.append(pre_position_ev, position[i])
mc_position_ev = pre_mc_position_ev.reshape(N_events,3)
position_ev = pre_position_ev.reshape(N_events,3)

for i in range(N_events):
    mc_radius_ev = np.append(mc_radius_ev,magnitude(mc_position_ev[i]))
    radius_ev = np.append(radius_ev, magnitude(position_ev[i]))

#Now, lets compare the radius of mc and reconstructed radius

dif_radi_ev = radius_ev - mc_radius_ev  # mc_radius_ev - radius_ev : useful to see where a bad reconstruction happened

N_events
#print(pmtid_ev)

7674

In [370]:
dif_radi

array([27.57906944, 27.57906944, 27.57906944, ..., 43.70641415,
       43.70641415, 43.70641415])

### Distribuitions of variables for a choosen eventID

In [371]:
# distribuitions of variables for a choosen eventID
var = [mc_radius_ev, radius_ev, dif_radi_ev ,time_res_ev, pmttime_ev]
i = 3
hist(var[i], 100, 'time_res_ev')

In [330]:
hist2D(dif_radi_ev, pmtid_ev, 100, 'dif_radi_ev','pmtid_ev')

In [334]:
hist2D(mc_radius_ev, dif_radi_ev, 30, 'mc_radius_ev','dif_radi_ev')

### Proof if it is right
Was verified using ROOT, same entries!
Open file .py in ROOT,
T-> GetEntries('evtid==14')

In [372]:
np.unique(pmt_type,return_counts=True) #([Nº PMT Type],[Nº of PMTs])

(array([ 1,  2,  3,  4,  5,  6,  7, 10]),
 array([9390,   91,   49,   22,    4,   32,    4,  136], dtype=int64))

### Looking for the index of the PMT by mean their ID and extract coordinates

---------------------------------------

Pruebas

In [286]:
# prueba
a = np.array([1,1,2,2,4])
(np.array(np.where(a == 2)))[0]

array([2, 3], dtype=int64)

In [287]:
#compare elements between arrays

x = array([1,2,3,4])
y = array([1,8,9,4,1,1])
print(np.in1d(y,x)) #y in x -less convenient because of dim(y)>dim(x)
print(np.in1d(x,y)) #x in y

[ True False False  True  True  True]
[ True False False  True]


-----------------------------------------

In [355]:
pmt_ids_idx = np.where(np.in1d(clean_pmt_id,pmtid_ev))[0] #index of PMTsID in pmtINFO in which is verified a hit in data

In [356]:
#index_pmt = array([index_pmt]).reshape((len(index)))
for i_idx in range(0,10):
    print(pmt_pos_xyz[i_idx])

[-99999. -99999. -99999.]
[ 5467.54    63.44 -6406.27]
[ 5615.16   266.8  -6251.09]
[ 5762.73   469.91 -6095.92]
[ 5910.5    673.23 -5940.54]
[ 7295.12  3775.89 -1812.97]
[ 5402.51   349.82 -6438.44]
[ 5550.13   553.18 -6283.26]
[ 5697.85   756.25 -6127.89]
[ 2380.63  -176.3  -8059.18]


In [357]:
#Extract coordinates of PMTs with hits

hitpmt_coord = []

#print(index_pmt)
for i in tqdm(pmt_ids_idx):
    #print(clean_pmt_id[i])
    hitpmt_coord.append(pmt_pos_xyz[clean_pmt_id[i]])

#print(len(hitpmt_coord))
hitpmt_coord = np.array(hitpmt_coord)#.reshape((len(hitpmt_coord),3))
hitpmt_coord[0][0]                              

100%|██████████████████████████████████████████████████████████████████████████| 4941/4941 [00:00<00:00, 215406.63it/s]


5467.54

In [291]:
clean_pmt_id

array([   1,    2,    3, ..., 3536, 3856, 8163])

In [292]:
pmt_ids_idx

array([   0,    1,    2, ..., 9390, 9391, 9393], dtype=int64)

In [293]:
len(pmt_ids_idx)

5337

In [294]:
np.shape(hitpmt_coord)

(5337, 3)

In [295]:
hitpmt_coord

array([[ 5467.54,    63.44, -6406.27],
       [ 5615.16,   266.8 , -6251.09],
       [ 5762.73,   469.91, -6095.92],
       ...,
       [ 1050.26,  7415.17,  3846.69],
       [-4287.58,  6152.61,  3796.11],
       [ 2545.4 , -7078.81,  3737.76]])

### PMT hit coord distribution

In [149]:
%matplotlib
fig = plt.figure(figsize = (10,10))
ax = plt.axes(projection='3d')
ax.grid()

x = hitpmt_coord[:,0]
y = hitpmt_coord[:,1]
z = hitpmt_coord[:,2]

ax.scatter(x, y, z, c = 'r', s = 1)
#plt.xlim(-10000,10000)
#plt.ylim(-10000,10000)

Using matplotlib backend: Qt5Agg


### Now, take this cleaned PMT id and lets clean data for a given event ID

- Usar PMTid_clean para buscar los indices de los pmts que dieron hit --> comparar (clean_pmt_id, hit_pmtid);
- Teniendo esa comparación extraer indices y extraer clean info. usando los indices;

#### Lets see the time residuals for the clean type PMT


In [373]:
time_res_clean = np.array([]) #Time residuals of clean data given a event
pmttime_clean = np.array([]) #time of pmt hit of in clean data given a event
pmtid_clean_ev = np.array([], dtype = np.int32) #clean pmt ID given a event
dif_radi_clean_ev = np.array([])  #clean radius diference

for i in tqdm(where(np.in1d(pmtid_ev, clean_pmt_id))[0]):
    pmtid_clean_ev = np.append(pmtid_clean_ev, pmtid_ev[i])
    time_res_clean = np.append(time_res_clean, hit_residual[i])
    pmttime_clean = np.append(pmttime_clean,hit_pmttime[i])
    dif_radi_clean_ev = np.append(dif_radi_clean_ev,dif_radi_ev[i])

100%|███████████████████████████████████████████████████████████████████████████| 7662/7662 [00:00<00:00, 20430.30it/s]


In [374]:
var_clean  = [time_res_clean, pmttime_clean, pmtid_clean_ev, dif_radi_clean_ev]
i = 0
hist(var_clean[i], 100,'time_res_clean')

# It shows how time residuals still getting high nevative values! Lets look the PMT ID which cause this ...

Hace sentido time residual a -600? ya verifique los cortes y desaparece el pico t t_res = -10.000 usando evento ID=11

In [324]:
hist2D(dif_radi_clean_ev, pmtid_clean_ev, 100, 'dif_radi_clean_ev','pmtid_clean_ev')

Cortar time residuar < -200 ns

In [360]:
r = time_res_clean < -200
for i in r:
    if i == True:
        print('here')

here
here
here
here
here
here
here
here


In [361]:
for i in pmtid_clean_ev:
    #if i == 9207:
    if i == 8080:
        print('here')

In [362]:
hist(pmttime_clean, 100, 'hit_pmttime')

No es culpa del pmt time visto que corta su valor negativo para el evento ID = 11

##### IDs and postion of PMTs giving t_res<-200 for this evet

In [375]:
pmtid_fail_ev = np.array([], dtype = np.int32) #id of pmt giving time residual < -200

fail_i = where(time_res_clean < -200)[0]

for i in fail_i:
    pmtid_fail_ev = np.append(pmtid_fail_ev, pmtid_clean_ev[i])
pmtid_fail_ev

array([5737, 8947, 9496, 6517,  407,  604,  330, 2440, 6491])

In [376]:
#now see where are these PMTs fails
fail_coord_xyz_ev = []

fail_pmtid_i = where(np.in1d(pmt_id, pmtid_fail_ev))[0] #pmt ID index in pmt info for which time residual < -200 for this evtID
#fail_pos_i

for i in fail_pmtid_i:
    fail_coord_xyz_ev.append(pmt_pos_xyz[pmt_id[i]])
    
fail_coord_xyz_ev = np.array(fail_coord_xyz_ev)
fail_coord_xyz_ev

array([[ 7623.35,  3569.09,  -290.55],
       [ 7852.84,   628.75,  2993.56],
       [ 5739.77,  3835.63, -4825.13],
       [  295.16,  8271.94, -1059.26],
       [-4547.3 , -5391.39, -4584.72],
       [-2088.14, -1833.55,  7950.91],
       [-3061.25,  -454.16,  7811.11],
       [ 5420.38, -1125.62,  6347.57],
       [ 8340.54,   118.36,  1047.87]])

In [377]:
fig = plt.figure(figsize = (10,10))
ax = plt.axes(projection='3d')
ax.grid()

x = fail_coord_xyz_ev[:,0]
y = fail_coord_xyz_ev[:,1]
z = fail_coord_xyz_ev[:,2]

ax.scatter(x, y, z, c = 'r', s = 10)
#plt.xlim(-10000,10000)
#plt.ylim(-10000,10000)

## Ultracleaning process after the clean_variables are gotten - pmtIDs of high negative time residuals for a given eventsID

#### The ultracleaning process variates with the eventID choosen, so the PMT info. derived will not be general!!

In [208]:
ultraclean_pmtid_ev = np.array([],dtype= np.int32) #PMT ID corresponding to time_residuals > -200
ultraclean_time_res = np.array([])  #Time residual values after the ultraclean process

ultra_i = where(time_res_clean > -200)[0] #Index from which will pick up  the PMT ID ultra clean (without time_residual < -200)

for i in ultra_i:
    ultraclean_pmtid_ev = np.append(ultraclean_pmtid_ev,pmtid_clean_ev[i])
    ultraclean_time_res = np.append(ultraclean_time_res, time_res_clean[i])
ultraclean_pmtid_ev

#AQUI NO HAY PROBLEMA

array([2579, 8519, 8484, ..., 2876, 8143, 2515])

In [209]:
hist(ultraclean_time_res, 100, 'ultraclean_time_res')

In [198]:
for i in ultra_i:
    if i < -200:
        print('here')

In [199]:
len(pmtid_clean_ev)
len(ultraclean_pmtid_ev)

7653

#### (ultraclean) PMT id

In [211]:
#extract the PMT ID ultra Clean for this event

ultra_PMTid = np.array([], dtype = np.int32)
index = where(np.in1d(clean_pmt_id, ultraclean_pmtid_ev))[0]

for i in index:
    ultra_PMTid = np.append(ultra_PMTid, clean_pmt_id[i])
ultra_PMTid

array([   1,    2,    3, ..., 2515, 3536, 8163])

## Lest cut and look for Cherenkov usign the ultraclean data

In [241]:
cut = 1 #cut for 10 ns in time residual

cev_pmt_id_ev = np.array([], dtype = np.int64) #PMT id of the cherenkov event 

cev_index = where(ultraclean_time_res <= 10)[0]

for i_dx in cev_index:
    cev_pmt_id_ev = np.append(cev_pmt_id_ev, ultraclean_pmtid_ev[i_dx])
    
cev_pmt_id_ev

array([2579, 8519, 8484, ..., 8472, 2171, 1518], dtype=int64)

In [242]:
cev_coord_xyz = []

for j in where(np.in1d(pmt_id, cev_pmt_id_ev))[0]:
    cev_coord_xyz.append(pmt_pos_xyz[pmt_id[j]])
cev_coord_xyz = np.array(cev_coord_xyz)
cev_coord_xyz

array([[ 5467.54,    63.44, -6406.27],
       [ 5762.73,   469.91, -6095.92],
       [ 7295.12,  3775.89, -1812.97],
       ...,
       [ 6497.03, -5090.15,  1679.06],
       [ 6196.91, -5402.28,  1664.71],
       [ 6078.68, -5666.4 ,  1149.76]])

### Now, lets see how the events looks -> See the coordinates of ultraclean PMTs giving hits

#### (ultraclean) PMT coordinates with hits and PMT id in PMT info

In [230]:
ultra_coordinates = [] #coordinates od PMTs after an ultracleaning

ultra_PMTid_i = where(np.in1d(pmt_id, ultraclean_pmtid_ev))[0] #index of ultraclean PMT ID in info PMT

for i in ultra_PMTid_i:
    ultra_coordinates.append(pmt_pos_xyz[pmt_id[i]])
    
ultra_coordinates = np.array(ultra_coordinates)

In [213]:
fig = plt.figure(figsize = (10,10))
ax = plt.axes(projection='3d')
ax.grid()

x = ultra_coordinates[:,0]
y = ultra_coordinates[:,1]
z = ultra_coordinates[:,2]

ax.scatter(x, y, z, c = 'r', s = 1)

In [243]:
fig = plt.figure(figsize = (10,10))
ax = plt.axes(projection='3d')
ax.grid()

x = cev_coord_xyz[:,0]
y = cev_coord_xyz[:,1]
z = cev_coord_xyz[:,2]

ax.scatter(x, y, z, c = 'r', s = 1)

#### (ultraclean) Time Residual

In [135]:
ultra_timeresidual_ev = np.array([])  #time residual after all the cuts of PMT ID - should give exactly time_res > -200

index = where(np.in1d(pmtid_clean_ev,ultra_PMTid))[0]

for i in index:
    ultra_time_residual_ev = np.append(ultra_time_residual_ev, time_res_clean[i])

In [136]:
hist(ultra_time_residual_ev, 100, 'ultra_time_residual_ev')

### Looks fine! Now lest cut in Time Residual values and see what we find (pray for me...)